# ESA risk list counterpart in Fink

In this notebook, we look for alerts in Fink that would correspond to identified risky asteroids from the ESA risk list (https://neo.ssa.esa.int/risk-list)

In [13]:
import requests
import pandas as pd
import io

def cross_match(oid: str, columns='i:objectId') -> int:
    """ Search alerts in Fink database corresponding to SSO designation `oid`
    
    Parameters
    ----------
    oid: str
        Name or designation for the object
    columns: str, optional
        Comma-separated column names to transfer.
        
    Returns
    ----------
    out: int
        Number of alerts matching `oid` in Fink database
    """
    r = requests.post(
        'https://api.fink-portal.org/api/v1/sso',
        json={
            'n_or_d': oid,
            'columns': columns,
            'output-format': 'json'
        }
    )
    if r.status_code != 200:
        return 0
    
    return len(pd.read_json(io.BytesIO(r.content)))

## Loading ESA data

We just downloaded the list from https://neo.ssa.esa.int/risk-list

In [14]:
risklist = pd.read_csv('riskList.csv')
risklist

,No.,Object designation,Diameter in m,Impact date/time in UTC,IP max,PS max,TS,Years,IP cum,PS cum,Vel. in km/s,In list since in d
0,1,2024YR4,50 - 100*,2032-12-22 14:51,1/1369,-1.67,1,2032,1/1041,-1.55,17.35,10
1,2,2023VD3,11 - 24*,2034-11-08 17:09,1/387,-2.65,0,2034-2098,1/387,-2.65,21.01,425
2,3,1979XB,400 - 900*,2056-12-12 21:38,1/4.27E6,-2.84,0,2056-2113,1/1.36E6,-2.72,27.54,5928
3,4,2008JL3,23 - 50*,2027-05-01 09:05,1/6711,-2.92,0,2027-2122,1/6211,-2.92,14.01,5928
4,5,2000SG344,27 - 60*,2071-09-16 00:54,1/1117,-3.19,0,2069-2122,1/354,-2.78,11.27,5928
...,...,...,...,...,...,...,...,...,...,...,...,...
1729,1730,2019HR2,5 - 10*,2097-04-23 16:19,1/2.37E10,-12.56,0,2097,1/2.37E10,-12.56,13.96,2081
1730,1731,2022CS7,7 - 15*,2114-02-18 18:10,1/6.58E10,-12.64,0,2114,1/6.58E10,-12.64,13.74,1051
1731,1732,2024EJ4,4 - 8*,2112-03-14 18:24,1/3.16E10,-12.89,0,2112,1/3.16E10,-12.89,13.04,298
1732,1733,2022BA7,1.9 - 4*,2117-01-27 05:38,1/7.81E9,-13.05,0,2117,1/7.81E9,-13.05,14.51,1068


## Crossmatch

We simply loop over the `Object designation` of the risk list, and see if there are corresponding alerts in Fink:

In [15]:
# takes a few minutes as we have over 600,000 sso in Fink.
oids = []
for oid in risklist['Object designation'].values:
    nobj = cross_match(oid)
    if nobj > 0:
        print(oid, '--> {} alert(s) found in Fink'.format(nobj))
        print('https://fink-portal.org/?query_type=SSO&n_or_d={}'.format(oid))
        print('https://fink-portal.org/api/v1/sso?n_or_d={}'.format(oid))
        oids.append(oid)

We found zero alert so far.

In [16]:
risklist[risklist['Object designation'].apply(lambda x: x in oids)]

,No.,Object designation,Diameter in m,Impact date/time in UTC,IP max,PS max,TS,Years,IP cum,PS cum,Vel. in km/s,In list since in d
